In [1]:
# !pip install optuna

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

from torch.autograd import Variable
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader

import joblib
import pickle
import optuna

warnings.filterwarnings('ignore')
%matplotlib inline

### 함수정의

In [3]:
def sliding_windows(data, lookback_length, forecast_length):

    x = []
    y = []
    
    for i in range(lookback_length, len(data) - forecast_length + 1):
        _x = data[(i-lookback_length) : i]
        _y = data[i : (i + forecast_length)]
        x.append(_x)
        y.append(_y)
    return np.array(x), np.array(y)


def get_data_loader(X, y, batch_size):

    x_train, x_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=False)

    train_ds = TensorDataset(torch.Tensor(x_train), torch.Tensor(y_train))
    train_dl = DataLoader(train_ds, batch_size = batch_size)

    val_ds = TensorDataset(torch.Tensor(x_val), torch.Tensor(y_val))
    val_dl = DataLoader(val_ds, batch_size = batch_size)

    input_size = x_train.shape[-1]

    return train_dl, val_dl, input_size

### Data loading && Preproces

In [5]:
scaler = MinMaxScaler()

data=pd.read_excel('./data_full.xlsx')
data = data[data["Date"].isin(pd.date_range('2016-01-04', '2019-11-30'))]
pd.date_range('2016-01-04', '2020-01-31')
pd.date_range('2016-01-04', '2020-03-31')
data.reset_index(drop=True, inplace=True)
data = data[['Date', 'REV OBD']]
scale_cols = ['REV OBD']

# Loockback_period & forecasting_period
max_prediction_length = 20
lookback_length = 60
training_data_max = len(data) - max_prediction_length

# 학습용 데이터
data_p = data.iloc[:training_data_max, :]
training_data = scaler.fit_transform(data_p[scale_cols])

#### Metric 생성을 위한 oot sample 정의

In [6]:
# max_prediction_length 만큼의 데이터는 예측 데이터와 비교를 위해 분리
# Training set에 없는 데이터로 구성
# Input과 output의 pair로 정의
x_for_metric = scaler.fit_transform(data[training_data_max -lookback_length : training_data_max][scale_cols])
y_for_metric = scaler.fit_transform(data[training_data_max:][scale_cols])


In [7]:
# LSTM은 1 step 뒤의 값만을 예측하므로, forecasting_period를 1로 두고 진행
x, y = sliding_windows(training_data, lookback_length, 1)

### Model 정의

In [8]:
class LSTM(nn.Module):
    def __init__(self, num_classes, input_size, hidden_size, num_layers):
        super(LSTM, self).__init__()
        self.num_classes = num_classes
        self.num_layers = num_layers
        self.input_size = input_size
        self.hidden_size = hidden_size
        
        self.lstm = nn.LSTM(input_size = input_size, hidden_size = hidden_size,
                            num_layers = num_layers, batch_first = True)
        
        self.fc = nn.Linear(hidden_size  * num_layers, num_classes)
        
    def forward(self, x):
        h_0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size, device = x.device))
        c_0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size, device = x.device))
        
        # Propagate input through LSTM
        ula, (h_out, _) = self.lstm(x, (h_0, c_0))
        h_out = h_out.view(-1, self.hidden_size * self.num_layers)
        out = self.fc(h_out)
        return out

In [9]:
def train(log_interval, model, train_dl, val_dl, optimizer, criterion, epoch):

    best_loss = np.inf
    for epoch in range(epoch):
        train_loss = 0.0
        model.train()
        for data, target in train_dl:

            if torch.cuda.is_available():
                data, target = data.cuda(), target.cuda()
                model = model.cuda()

            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target) # mean-squared error for regression
            loss.backward()
            optimizer.step()
            train_loss += loss.item()

        # validation
        valid_loss = 0.0
        model.eval()
        for data, target in val_dl:

            if torch.cuda.is_available():
                data, target = data.cuda(), target.cuda()

            output = model(data)         
            loss = criterion(output, target)
            valid_loss += loss.item()

        if ( epoch % log_interval == 0 ):
            print(f'\n Epoch {epoch} \t Training Loss: {train_loss / len(train_dl)} \t Validation Loss: {valid_loss / len(val_dl)} \n')

        if best_loss > (valid_loss / len(val_dl)):
            print(f'Validation Loss Decreased({best_loss:.6f}--->{(valid_loss / len(val_dl)):.6f}) \t Saving The Model')
            best_loss = (valid_loss / len(val_dl))
            torch.save(model.state_dict(), 'lstm_saved_model.pth')

    return best_loss


def smape(a, f):
    return 1/len(a) * np.sum(2 * np.abs(f-a) / (np.abs(a) + np.abs(f))*100)

In [17]:
aa = x_for_metric
tmp = np.append( np.expand_dims(aa[1:, :], 0), np.expand_dims(y_for_metric[2, :], (0,2)), axis=1)
tmp.shape

(1, 60, 1)

In [22]:
def objective(trial):

    cfg = { 
            'batch_size' : trial.suggest_categorical('batch_size',[64, 128, 256, 512]), # [64, 128, 256]
            'learning_rate' : trial.suggest_loguniform('learning_rate', 1e-3, 1e-1), #trial.suggest_loguniform('learning_rate', 1e-2, 1e-1), # learning rate을 0.01-0.1까지 로그 uniform 분포로 사용
            'hidden_size': trial.suggest_categorical('hidden_size',[16, 32, 64, 128, 256, 512, 1024]),
            'num_layers': trial.suggest_int('num_layers', 1, 5, 1),       
        }

    torch.manual_seed(42)

    log_interval = 5
    num_classes = 1 # parameter에서 빼서 상수로 설정
    num_epochs = 10 # parameter에서 빼서 상수로 설정

    train_dl, val_dl, input_size = get_data_loader(x, y,  cfg['batch_size'])
    
    model = LSTM(
        num_classes = num_classes, 
        input_size = input_size, 
        hidden_size = cfg['hidden_size'], 
        num_layers = cfg['num_layers']
    )
    
    if torch.cuda.is_available():
        model = model.cuda()
        
    optimizer = optim.Adam(model.parameters(), lr=cfg['learning_rate'])
    criterion = torch.nn.MSELoss()
    best_loss = train(log_interval, model, train_dl, val_dl, optimizer, criterion, num_epochs)

    print('best loss for the trial = ', best_loss)
    predict_data = []
    # 여기서 x는 (sample, lookback_length, 1)의 크기를 지님. 따라서, 제일 앞의 시점을 제거하려면, x[:, -1, :]이 되어야 함
    x_pred = np.expand_dims(x_for_metric, 0)  # Inference에 사용할 lookback data를 x_pred로 지정. 앞으로 x_pred를 하나씩 옮겨 가면서 inference를 할 예정

    for j, i in enumerate(range(max_prediction_length)):

        # feed the last forecast back to the model as an input
        x_pred = np.append( x_pred[:, 1:, :], np.expand_dims(y_for_metric[j, :], (0,2)), axis=1)
        xt_pred = torch.Tensor(x_pred)

        if torch.cuda.is_available():
            xt_pred = xt_pred.cuda()
        # generate the next forecast
        yt_pred = model(xt_pred)
        # tensor to array
        # x_pred = xt_pred.cpu().detach().numpy()
        y_pred = yt_pred.cpu().detach().numpy()

        # save the forecast
        predict_data.append(y_pred)

    # transform the forecasts back to the original scale
    predict_data = np.array(predict_data).reshape(-1, 1)
    SMAPE = SMAPE(y_for_metric, predict_data)
    
    print(f' \nSMAPE : {SMAPE}')


    return SMAPE

In [23]:
sampler = optuna.samplers.TPESampler()
#   sampler = optuna.samplers.SkoptSampler()

# model.load_state_dict(torch.load('lstm_saved_model.pth'))
    
study = optuna.create_study(sampler=sampler, direction='minimize')
study.optimize(objective, n_trials= 5)

[I 2022-06-18 16:02:23,616] A new study created in memory with name: no-name-7461789a-55e8-4c33-a00b-a3d17ce22581



 Epoch 0 	 Training Loss: 0.11291885823011398 	 Validation Loss: 0.04117981716990471 

Validation Loss Decreased(inf--->0.041180) 	 Saving The Model
Validation Loss Decreased(0.041180--->0.023098) 	 Saving The Model
Validation Loss Decreased(0.023098--->0.016731) 	 Saving The Model

 Epoch 5 	 Training Loss: 0.5707897007465362 	 Validation Loss: 0.2192603498697281 



[I 2022-06-18 16:02:26,398] Trial 0 finished with value: 58.02022040725072 and parameters: {'batch_size': 256, 'learning_rate': 0.008231698701227924, 'hidden_size': 256, 'num_layers': 4}. Best is trial 0 with value: 58.02022040725072.


best loss for the trial =  0.01673095254227519
 
SMAPE : 58.02022040725072

 Epoch 0 	 Training Loss: 0.04637186957851929 	 Validation Loss: 0.0216861542314291 

Validation Loss Decreased(inf--->0.021686) 	 Saving The Model
Validation Loss Decreased(0.021686--->0.020385) 	 Saving The Model
Validation Loss Decreased(0.020385--->0.019178) 	 Saving The Model

 Epoch 5 	 Training Loss: 0.025722071966704202 	 Validation Loss: 0.019340337067842484 

Validation Loss Decreased(0.019178--->0.019158) 	 Saving The Model
Validation Loss Decreased(0.019158--->0.019020) 	 Saving The Model
Validation Loss Decreased(0.019020--->0.018918) 	 Saving The Model


[I 2022-06-18 16:02:28,960] Trial 1 finished with value: 65.06253435216556 and parameters: {'batch_size': 64, 'learning_rate': 0.0027539087138622894, 'hidden_size': 64, 'num_layers': 4}. Best is trial 0 with value: 58.02022040725072.


...

best loss for the trial =  0.01655278044442336
 
SMAPE : 67.91946493770153

 Epoch 0 	 Training Loss: 1.9409560166737612 	 Validation Loss: 0.5459761917591095 

Validation Loss Decreased(inf--->0.545976) 	 Saving The Model
Validation Loss Decreased(0.545976--->0.085423) 	 Saving The Model
Validation Loss Decreased(0.085423--->0.044357) 	 Saving The Model
Validation Loss Decreased(0.044357--->0.034689) 	 Saving The Model

 Epoch 5 	 Training Loss: 0.033054652459481186 	 Validation Loss: 0.021773317083716392 

Validation Loss Decreased(0.034689--->0.021773) 	 Saving The Model
Validation Loss Decreased(0.021773--->0.018333) 	 Saving The Model
Validation Loss Decreased(0.018333--->0.018072) 	 Saving The Model


[I 2022-06-18 16:02:32,219] Trial 3 finished with value: 64.63681781193611 and parameters: {'batch_size': 64, 'learning_rate': 0.012339588600570424, 'hidden_size': 256, 'num_layers': 3}. Best is trial 0 with value: 58.02022040725072.


best loss for the trial =  0.01807173080742359
 
SMAPE : 64.63681781193611

 Epoch 0 	 Training Loss: 0.07857944294810296 	 Validation Loss: 0.023710521403700113 

Validation Loss Decreased(inf--->0.023711) 	 Saving The Model
Validation Loss Decreased(0.023711--->0.016537) 	 Saving The Model
Validation Loss Decreased(0.016537--->0.015468) 	 Saving The Model

 Epoch 5 	 Training Loss: 0.024748937413096427 	 Validation Loss: 0.016025313176214695 



[I 2022-06-18 16:02:33,413] Trial 4 finished with value: 68.26887900034627 and parameters: {'batch_size': 256, 'learning_rate': 0.008916999467114078, 'hidden_size': 128, 'num_layers': 4}. Best is trial 0 with value: 58.02022040725072.


Validation Loss Decreased(0.015468--->0.015364) 	 Saving The Model
best loss for the trial =  0.015364142134785652
 
SMAPE : 68.26887900034627
